## Notes
Calculating all conformers for molecules can take a long time depending on how many OSDAs (typically about 12 hours for 300 molecules). This task can easily run in parallel. The easiest way to accomplish this is to make several copies of this script notebook and load in different sections of your OSDA list to each one. This code borrows heavily from Wicker et al. Beyond Rotatable Bond  Counts: Capturing 3D Conformational Flexibility in a Single Descriptor. J. Chem. Inf. Model. 2016. 10.1021/acs.jcim.6b00565

In [1]:
# Change to the top directory
%cd ..

/home/OSDA_Generator


In [18]:
import numpy as np
import pandas as pd
import json
import time
import pickle
from rdkit import Chem
from rdkit.Chem import rdFreeSASA, Descriptors, Descriptors3D, AllChem, Draw
from rdkit.Chem.rdForceFieldHelpers import MMFFOptimizeMolecule

In [3]:
# load in data
data = pd.read_excel('data/Jensen_et_al_CentralScience_OSDA_Zeolite_data.xlsx', engine='openpyxl')

In [19]:
# specify the SMILES strings, can specify a user-defined list, set a path to a pickle file (usually)
# for generated smiles, or calculate from the data set
all_smiles = []
smiles = [] # user specified smiles
path_to_file = '' # path to pickle file containing OSDAs, often from generated OSDAs
if smiles:
    print('Using user defined OSDAs')
    for s in smiles:
        can = Chem.MolToSmiles(Chem.MolFromSmiles(s), canonical=True)
        if can not in all_smiles:
            all_smiles.append(can)
elif path_to_file != '':
    print('Reading SMILES from', path_to_file)
    curr = pickle.load(open(path_to_file, 'rb'))
    for c in curr:
        can = Chem.MolToSmiles(Chem.MolFromSmiles(c), canonical=True)
        if can not in all_smiles:
            all_smiles.append(can)
else:
    print('Using all SMILES from data set')
    for i, row in data.iterrows():
        if pd.isnull(row['smiles']):
            continue
        else:
            curr = row['smiles'].split(' + ')
            for smile in curr:
                m = Chem.MolFromSmiles(smile)
                if m is not None:
                    can = Chem.MolToSmiles(m, canonical=True)
                    if can not in all_smiles:
                        all_smiles.append(can)
print(len(all_smiles))

Reading SMILES from data/generated_osdas.pickle
21


In [24]:
# Specify dictionary to save conformers
all_osda_features = dict()

In [25]:
# calulate the conformers
number_of_conformers = 2000  # highest possible number of conformers
start = time.time()
for j, smile in enumerate(all_smiles):
    if smile in all_osda_features.keys():
        continue # This stops from overwriting if code gets interrupted
    print(j, smile)
    all_osda_features[smile] = dict()
    all_osda_features[smile]['volumes'] = []
    all_osda_features[smile]['energies'] = []
    all_osda_features[smile]['whim'] = []
    m = Chem.MolFromSmiles(s)
    m2 = Chem.AddHs(m)
    conformers = AllChem.EmbedMultipleConfs(m2, numConfs=number_of_conformers, pruneRmsThresh=0.5, numThreads=100)
    optimised_and_energies = AllChem.MMFFOptimizeMoleculeConfs(m2, maxIters=600, numThreads=100, nonBondedThresh=100.0)
    ids = []
    all_rms = []
    for c in conformers:
        if optimised_and_energies[c][0] != 0:
            continue
        dont_add = False
        for c2 in conformers[(c+1):]:
            rms = AllChem.GetConformerRMS(m2, c, c2)
            if rms < 1.0:
                dont_add = True
        if not dont_add:
            ids.append(c)
    all_osda_features[smile]['rms'] = all_rms
    for i in ids:
        all_osda_features[smile]['volumes'].append(AllChem.ComputeMolVolume(m2, conformers[i]))
        all_osda_features[smile]['energies'].append(optimised_and_energies[i][1])
        all_osda_features[smile]['whim'].append(Chem.rdMolDescriptors.CalcWHIM(m2, conformers[i]))
    print(len(all_osda_features[smile]['volumes']),len(all_osda_features[smile]['energies']),len(all_osda_features[smile]['whim']))
    print(round(time.time()-start,3))
    start= time.time()

0 CC[N+]1(C)C2CCCC1CC2
16 16 16
99.895
1 C[N+]1(C)CCCC2CCCC21
17 17 17
101.076
2 C[N+]1(C)CC2C3C=CC(CC3)C2C1
18 18 18
100.111
3 C[N+]1(C)CCC2CCCC2C1
14 14 14
101.018
4 C[N+]1(C)CC2CC3CC(C2)CC1C3
15 15 15
101.16
5 C[N+](C)(C)C1C2CCCC1CCC2
17 17 17
102.224
6 C[N+]1(C)CCCC2CCCCC21
15 15 15
98.275
7 CC[N+]1(C)CC2CC3CC(C2)CC1C3
18 18 18
101.202
8 CC[N+]1(C)CCCC(C)(C)C1
16 16 16
99.71
9 CC1(C)CCC[N+](C)(C)C1
16 16 16
99.968
10 C[N+](C)(C)C12CC3CC(CC(C3)C1)C2
17 17 17
101.053
11 C[N+]1(C)CCCC2CCCCC2C1
14 14 14
102.493
12 C[N+]12CCCCC1C1CCC(C1)C2
15 15 15
99.833
13 CC1CC(C)C[N+](C)(C)C1
14 14 14
102.585
14 CC1(C)C2CCC[N+](C)(C)C1C2
19 19 19
105.858
15 CC[N+]1(C)CC2CCC(CC2)C1
14 14 14
103.507
16 CC1CCC[N+]12CCCC(C)(C)C2
16 16 16
106.114
17 CC[N+]1(C)CC2CCC(C2)C1
16 16 16
106.013
18 CC1(C)CC2CC(C)(C1)C[N+]2(C)C
15 15 15
103.054
19 C1CC[N+]2(C1)CC1CCC(CC1)C2
19 19 19
105.755
20 CC[N+]1(C)C2CCCC1CCC2
17 17 17
100.801


In [16]:
# use json to save conformers
with open('data/osda_conformers.json', 'w') as fp:
    json.dump(all_osda_features, fp)